# Projeto 1 - Ciência dos Dados

Nome: Enzo Cardoso de Almeida Santos Neto

Nome: Luis Costa

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
import math
import os.path
import pandas as pd
import json
from random import shuffle
import re 
import matplotlib.pyplot as plt
import emoji
from emoji  import UNICODE_EMOJI

### Caso de erro ao rodar o Import Emoji, por favor digitar no Anaconda Prompt o seguinte comando:

 pip install emoji

___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

O nosso produto analisado foi um Iphone XR, lançado no ano de 2018 pela Apple. Ele é considerado um smartphone topo de linha e ate hoje é muito utilizado e vendido, tanto em territorio nacional, quanto internacional.
Para fazer a classificação, primeiro filtramos por linguas, onde pegamos apenas os tweets em Português. Em seguida, classificamos como relevante os tweets que eram pertinentes e falavam sobre o dispositivo, tanto bem quanto mal.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [2]:
#Lendo o excel para o treinamento do código
base_total = pd.read_excel("base_iphoneXR.xlsx", sheet_name="Treinamento")

#Série de todos os dados + sua tabela absoluta
serie_tudo = pd.Series(base_total.Treinamento)
tabela_tudo_absoluta = serie_tudo.value_counts()

In [3]:
#Função para limpar os sinais básicos e separa emojis.
def remocao_de_sinais(texto):
    import string
    pontuacao = '[!-.:?;]'
    padrao = re.compile(pontuacao)
    text_subbed = re.sub(padrao, ' ', texto)
    letras=[]
    for letra in text_subbed:  
        letras.append(letra)
    i=1
    espaço=" "
    frase_separando_emoji=""
    while i<= (len(letras)-1):        
        if letras[i] in UNICODE_EMOJI:
            frase_separando_emoji+=(espaço)
            frase_separando_emoji+=letras[i-1]
            frase_separando_emoji+=(espaço)
            frase_separando_emoji+=(letras[i])
            letras.remove(letras[i])
            i+=1
        else:
            frase_separando_emoji+=(letras[i-1])
            i+=1
    return frase_separando_emoji


In [4]:
#limpando os dados de treinamento
tweets_relevantes = open("relevante.txt", "r", encoding="utf8")
relevante_iphone = remocao_de_sinais((tweets_relevantes.read()).lower())

tweets_irrelevantes = open("irrelevante.txt", "r", encoding="utf8")
irrelevante_iphone = remocao_de_sinais((tweets_irrelevantes.read()).lower())

In [5]:
#tabelas relativas e absolutas
relevante_relativa = (pd.Series(relevante_iphone.split())).value_counts(True)
relevante_absoluta = (pd.Series(relevante_iphone.split())).value_counts(False)

irrelevante_relativa = (pd.Series(irrelevante_iphone.split())).value_counts(True)
irrelevante_absoluta = (pd.Series(irrelevante_iphone.split())).value_counts(False)

In [6]:
#probabilidades
probabilidade_relevante = relevante_absoluta.sum()/tabela_tudo_absoluta.sum()
probabilidade_irrelevante = irrelevante_absoluta.sum()/tabela_tudo_absoluta.sum()

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [7]:
#definindo a função Laplace
def laplace(palavra, tabela1, tabela2):
    if palavra in tabela1:
        x = tabela1[palavra]
    else:
        x = 0
        
    if palavra in tabela2:
        y = tabela2[palavra]
    else:
        y = 0 
    
    prob_relevante = (x + 1)/ (len(tabela1) + len(set(tabela1+tabela2)))
    prob_irrelevante = (y + 1)/ (len(tabela2) + len(set(tabela1+tabela2)))
    return [prob_relevante, prob_irrelevante]    

In [8]:
#lendo o excel com os tweets de teste.
base_teste = pd.read_excel("base_iphoneXR.xlsx",sheet_name="Teste")
series_tweets = pd.Series(list(base_teste.Teste))

In [9]:
#limpando os tweets do teste
tweets_prontos = []
for tweet in series_tweets:
    tweets_prontos.append(remocao_de_sinais(tweet.lower()))


#classificador
classificacao = {}
lista_ok = []
for frase in tweets_prontos:
    palavras_da_frase = list(frase.split())
    relevante = 1
    irrelevante = 1
    
    for palavra in palavras_da_frase:
        
        # caso esteja nas duas categorias
        if palavra in relevante_absoluta and \
        palavra in irrelevante_absoluta:
            relevante *= relevante_absoluta[palavra]
            irrelevante *= irrelevante_absoluta[palavra]
        
        #caso esteja somente em relevante
        elif palavra in relevante_absoluta and \
        palavra not in irrelevante_absoluta:
            relevante *= relevante_absoluta[palavra]
            irrelevante *= laplace(palavra,relevante_absoluta,irrelevante_absoluta)[1]
        
        #caso esteja somente em irrelevante
        elif palavra in irrelevante_absoluta and \
        palavra not in relevante_absoluta:
            relevante *= laplace(palavra,relevante_absoluta,irrelevante_absoluta)[0]
            irrelevante *= irrelevante_absoluta[palavra]
        
        #caso nao esteja em nenhuma categoria
        else:
            relevante *= laplace(palavra,relevante_absoluta,irrelevante_absoluta)[0]
            irrelevante *= laplace(palavra,relevante_absoluta,irrelevante_absoluta)[1]
    
    #probabilidade da frase(tweet) ser relevante e irrelevante
    prob_relevante = probabilidade_relevante*relevante
    prob_irrelevante = probabilidade_irrelevante*irrelevante
    
    #comparacao das probabilidades
    if prob_relevante > prob_irrelevante:
        classificacao[frase] = 0
        lista_ok.append(0)
    else:
        classificacao[frase] = 1
        lista_ok.append(1)
        
classificados = pd.Series(classificacao)

E:\ANACONDA\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: overflow encountered in longlong_scalars
E:\ANACONDA\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: overflow encountered in longlong_scalars
E:\ANACONDA\lib\site-packages\ipykernel_launcher.py:33: RuntimeWarning: overflow encountered in longlong_scalars
E:\ANACONDA\lib\site-packages\ipykernel_launcher.py:26: RuntimeWarning: overflow encountered in longlong_scalars


In [17]:
#comparando o teste com a classificação 
for tweet in classificacao:
    dic={'Tweet': tweets_prontos, 'Resultado Teste': lista_ok, 'Classificação': base_teste["Classe"]}

In [18]:
tabela_classificacao= pd.DataFrame(dic, columns=['Tweet','Resultado Teste','Classificação'])

In [54]:
acertos = 0
errados = 0

#varrendo as respostas
for teste, certo in zip(lista_ok, base_teste["Classe"]):
    if teste == certo:
        acertos += 1
    else:
        errados += 1
        
porcentagem_acertos = (acertos / 81) *100
porcentagem_errados = (errados / 81) *100
print("Porcentagem de acertos = {}%".format(porcentagem_acertos))
print("Porcentagem de erros = {}%".format(porcentagem_errados))

Porcentagem de acertos = 41.9753086419753%
Porcentagem de erros = 56.79012345679012%


In [56]:
verdadeiros_positivo=0
falsos_positivos=0
verdadeiros_negativos=0
falsos_negativos=0

#relevante=0 e irrelevante=1
for resultado_teste,resultado_certo in zip(lista_ok,base_teste["Classe"]):
    
    if resultado_certo == 0 and resultado_teste == 0:
        verdadeiros_positivo += 1
        
    elif resultado_certo == 1 and resultado_teste == 0:
        falsos_positivos += 1
        
    elif resultado_certo == 1 and resultado_teste == 1:
        verdadeiros_negativos += 1
        
    elif falsos_negativos == 0 and resultado_teste == 1:
        falsos_negativos += 1
    
Porcentagem_verdadeiros_positivo = (verdadeiros_positivo / 81) *100
Porcentagem_falsos_positivos = (falsos_positivos / 81) *100
Porcentagem_verdadeiros_negativos = (verdadeiros_negativos / 81) *100
Porcentagem_falsos_negativos = (falsos_negativos / 81) *100

print("Porcentagem de verdadeiros positivos: {}%".format(Porcentagem_verdadeiros_positivo))
print("Porcentagem de falsos positivos: {}%".format(Porcentagem_falsos_positivos))
print("Porcentagem de verdadeiros negativos: {}%".format(Porcentagem_verdadeiros_negativos))
print("Porcentagem de falsos negativos: {}%".format(Porcentagem_falsos_negativos))

Porcentagem de verdadeiros positivos: 35.80246913580247%
Porcentagem de falsos positivos: 38.2716049382716%
Porcentagem de verdadeiros negativos: 6.172839506172839%
Porcentagem de falsos negativos: 1.2345679012345678%


___
### Concluindo

In [76]:
acertos_sem_laplace=39.43587829098304
V_P_sem_laplace= 35.76855895196506
F_P_sem_laplace= 40.04803493449782
V_N_sem_laplace= 5.327510917030565
F_N_sem_laplace= 1.13668122270742354

porcentagem_certos_sem_separar_emoji= 38.87290969899666
Porcentagem_verdadeiros_positivo_sem_separar_emoji= 34.89975290655021
Porcentagem_falsos_positivos_sem_separar_emoji= 40.97207482073615
Porcentagem_verdadeiros_negativos_sem_separar_emoji= 4.970829694323145
Porcentagem_falsos_negativos_sem_separar_emoji= 1.13668122270742354

dados = {
'Métodos': ['Sem Laplace', 'Com Laplace', 'Laplace + separação de emoji'],
'Porcentagem de acertos(%)': [acertos_sem_laplace,porcentagem_certos_sem_separar_emoji,porcentagem_acertos],
'Porcentagem de verdadeiros positivos(%)':[V_P_sem_laplace,Porcentagem_verdadeiros_positivo_sem_separar_emoji,Porcentagem_verdadeiros_positivo],
'Porcentagem de falsos positivos(%)':[F_P_sem_laplace,Porcentagem_falsos_positivos_sem_separar_emoji,Porcentagem_falsos_positivos],
'Porcentagem de verdadeiros negativos(%)':[V_N_sem_laplace,Porcentagem_verdadeiros_negativos_sem_separar_emoji,Porcentagem_verdadeiros_negativos],
'Porcentagem de falsos negativos(%)':[F_N_sem_laplace,Porcentagem_falsos_negativos_sem_separar_emoji,Porcentagem_falsos_negativos]
}

In [77]:
tabela_SEMxCOMxEMOJI = pd.DataFrame(dados,\
                                    columns=['Métodos', 'Porcentagem de acertos(%)',\
                                             'Porcentagem de verdadeiros positivos(%)',\
                                             'Porcentagem de falsos positivos(%)',\
                                             'Porcentagem de verdadeiros negativos(%)',\
                                             'Porcentagem de falsos negativos(%)'])

In [78]:
tabela_SEMxCOMxEMOJI

,Métodos,Porcentagem de acertos(%),Porcentagem de verdadeiros positivos(%),Porcentagem de falsos positivos(%),Porcentagem de verdadeiros negativos(%),Porcentagem de falsos negativos(%)
0,Sem Laplace,39.435878,35.768559,40.048035,5.327511,1.136681
1,Com Laplace,38.872910,34.899753,40.972075,4.970830,1.136681
2,Laplace + separação de emoji,41.975309,35.802469,38.271605,6.172840,1.234568


   No primeiro instante, fizemos sem utilizar o Laplace, porém, como o resultado obtido foi menor do que o esperado (apagamos o arquivo),resolvemos utilizar o laplace para tentar aumentar o nosso resultado. Sem sucesso com o Laplace, fomos atrás de
outro metodo para melhorar nossos resultados. Baixamos outra biblioteca onde separamos emojis. Os resultados obtidos foram 
melhores, porém continuaram muito proximos dos outros, fazendo assim com que pudessemos concluir que o modelo por mais que tenha um acerto de 40% aproximadamente, não é suficente para ser um modelo confiavel de classificação. Outro jeito de aperfeiçoar e aumentar nosso resultado poderia ser, categorizar em mais grupos ou pegar uma base de testes maior.

   Poderiamos usar a teoria de classificação de Naive Bayes para calcular por exemplo a probabilidade de, ao lançarmos um produto no mercado, qual é o genero mais sucetivel a comprar esse produto de acordo com as respostas de uma survey.

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**